# Test timeout feature

## Table of content (ToC)<a class="anchor" id="TOC"></a>
* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load the morphkit library</a>
* <a href="#bullet3">3 - Load the morphkit library</a>
* <a href="#bullet4">4 - Perform some tests</a>
* <a href="#bullet5">5 - Required libraries</a>
* <a href="#bullet6">6 - Test timeout to non-existing API endpoint</a>
* <a href="#bullet7">7 - Notebook version</a>

#  1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

This Jupyter Notebook performs some test on the timeout feature.

# 2 - Resolve dependencies <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The morphkit package depends on beta code package.

In [8]:
!pip install beta_code

# 3 - Load the morphkit library <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

This is my own library (under development) available via [github.com/tonyjurg/morphkit](https://github.com/tonyjurg/morphkit). This is developed to simplify interactions with the Morpheus service running on my Docker container.

In [12]:
import sys
sys.path.insert(0, r"..\..\morphkit")    # relative to notebook dir
import morphkit

morphkit loaded


# 4 - Verify the changed functions <a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

We will first print the docstrings of the changed functions by calling `help()`.

In [15]:
help(morphkit.get_word_blocks) 

Help on function get_word_blocks in module morphkit.get_word_blocks:

get_word_blocks(word_beta: str, api_endpoint: str, language: str = 'greek', output: str = 'full', debug: bool = False, timeout: Union[int, float, NoneType] = None, retry_attempts: Optional[int] = None, retry_delay: Optional[float] = None) -> str
    Retrieve the raw word blocks data for a given beta-code word from a Morpheus endpoint.

    Args:
    -----

        :word_beta (str):        The input word in beta-code format to look up.
                                 Backslashes in the input string need to be escaped: e.g., 'a)nh/r' -> 'a)nh/r\'

        :api_endpoint (str):  IP adress & port of the  Morpheus API endpoint (e.g., '192.168.0.5:1315').

        :language (str):    Optional argument. Defaults to `greek`. Sets the language of the word to analyse. It can be set to `greek` or `latin`.

        :output {str}:      Optional argument. Defaults to `full`. Output format of the Analytic block. Either `full` for t

In [16]:
help(morphkit.analyse_word_with_morpheus)

Help on function analyse_word_with_morpheus in module morphkit.analyse_word_with_morpheus:

analyse_word_with_morpheus(word_beta: str, api_endpoint: str, language: str = 'greek', add_pos: bool = True, add_morph: bool = True, debug: bool = False, timeout: Optional[float] = None, retry_attempts: Optional[int] = None, retry_delay: Optional[float] = None) -> Dict[str, Any]
    Query the Morpheus morphological analyser for a Greek word in Betacode and parse its analyses.

    Args:
    -----

        :word_beta (str):     The input word in beta-code format to look up.
                              Backslashes in the input need to be escaped: e.g., 'a)nh/r' -> 'a)nh/r\'.

        :api_endpoint (str):  IP adress & port of the  Morpheus API endpoint (e.g., 192.168.0.5:1315).

        :language (str):      Optional argument. Defaults to `greek`. The other option is 'latin'.
                              If set to 'latin' no POS and morph field will be added.

        :add_pos (bool):      Optio

# 5 - Perform some regression tests <a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

First, we will perform some regression testing (backward compatibility testing).

In [23]:
# convert unicode greek to betacode
import beta_code
bc_word=beta_code.greek_to_beta_code(u'Δία')
print (bc_word)
api_endpoint="10.0.1.156:1315" # IP/port
import pprint as pp
raw_text=morphkit.get_word_blocks(bc_word,api_endpoint,debug=True)
print(raw_text)
blocks=morphkit.split_into_raw_blocks(raw_text)
all_parses = []
for block in blocks:
    raw_beta, parses = morphkit.parse_word_block(block,debug=True)
    all_parses.append(parses)
    pp.pprint(parses)

*di/a
[get_word_blocks] Sending GET request: http://10.0.1.156:1315/greek/%2Adi%2Fa?opts=d?opts=n
[get_word_blocks] Received status code: 200
[get_word_blocks] Response time: 0.025s
[get_word_blocks] Request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
[get_word_blocks] Response headers: {'Content-Type': 'text/html;charset=utf-8', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Length': '1759'}
[get_word_blocks] Response snippet (max 100 bytes):

:raw *di/a

:workw *di/a_
:lem *di/h
:prvb 				
:aug1 				
:stem *di	 fem sg			h_hs
:suff 				
:end...

:raw *di/a

:workw *di/a_
:lem *di/h
:prvb 				
:aug1 				
:stem *di	 fem sg			h_hs
:suff 				
:end a_	 fem nom/voc sg	doric aeolic		h_hs

:raw *di/a

:workw *di/a_
:lem *di/h-pl
:prvb 				
:aug1 				
:stem *di	 fem dual			h_hs
:suff 				
:end a_	 fem nom/voc/acc dual			h_hs

:raw

In [18]:
import pprint as pp
for [parse] in all_parses:  #[] since list of dictionaries
    parse['pos']=(morphkit.analyse_pos(parse,debug=True))
    pp.pprint(parse)

[analyse_pos] betacode *di/a:  word=None ordered_items=['h_hs', 'h_hs']
[analyse_pos] case or gender 🠢 noun ~ N-
{'case': ['nom', 'voc'],
 'dialects': ['doric', 'aeolic'],
 'end_bc': 'a_',
 'end_codes': ['h_hs'],
 'end_uc': 'α—',
 'gender': 'fem',
 'lem_base_bc': '*di/h',
 'lem_base_uc': 'Δίη',
 'lem_full_bc': '*di/h',
 'lem_full_uc': 'Δίη',
 'number': 'sg',
 'pos': 'noun',
 'raw_bc': '*di/a',
 'raw_uc': 'Δία',
 'stem_bc': '*di',
 'stem_codes': ['h_hs'],
 'stem_gender': 'fem',
 'stem_number': 'sg',
 'stem_uc': 'Δι',
 'workw_bc': '*di/a_',
 'workw_uc': 'Δία—'}
[analyse_pos] betacode *di/a:  word=None ordered_items=['h_hs', 'h_hs']
[analyse_pos] case or gender 🠢 noun ~ N-
{'case': ['nom', 'voc', 'acc'],
 'end_bc': 'a_',
 'end_codes': ['h_hs'],
 'end_uc': 'α—',
 'gender': 'fem',
 'lem_base_bc': '*di/h',
 'lem_base_uc': 'Δίη',
 'lem_full_bc': '*di/h-pl',
 'lem_full_uc': 'Δίη-πλ',
 'lem_pl_suff': '1',
 'number': 'dual',
 'pos': 'noun',
 'raw_bc': '*di/a',
 'raw_uc': 'Δία',
 'stem_bc': '*di'

In [19]:
import pprint as pp
for [parse] in all_parses:  #[] since list of dictionaries
    parse['morph']=(morphkit.analyse_morph_tag(parse,debug=True))
    pp.pprint(parse)

[analyse_morph_tag] Called with betacode *di/a: noun/adjective/article 🠢 N-NSF-A/N-VSF-A
{'case': ['nom', 'voc'],
 'dialects': ['doric', 'aeolic'],
 'end_bc': 'a_',
 'end_codes': ['h_hs'],
 'end_uc': 'α—',
 'gender': 'fem',
 'lem_base_bc': '*di/h',
 'lem_base_uc': 'Δίη',
 'lem_full_bc': '*di/h',
 'lem_full_uc': 'Δίη',
 'morph': 'N-NSF-A/N-VSF-A',
 'number': 'sg',
 'pos': 'noun',
 'raw_bc': '*di/a',
 'raw_uc': 'Δία',
 'stem_bc': '*di',
 'stem_codes': ['h_hs'],
 'stem_gender': 'fem',
 'stem_number': 'sg',
 'stem_uc': 'Δι',
 'workw_bc': '*di/a_',
 'workw_uc': 'Δία—'}
[analyse_morph_tag] Called with betacode *di/a: noun/adjective/article 🠢 N-NDF/N-VDF/N-ADF
{'case': ['nom', 'voc', 'acc'],
 'end_bc': 'a_',
 'end_codes': ['h_hs'],
 'end_uc': 'α—',
 'gender': 'fem',
 'lem_base_bc': '*di/h',
 'lem_base_uc': 'Δίη',
 'lem_full_bc': '*di/h-pl',
 'lem_full_uc': 'Δίη-πλ',
 'lem_pl_suff': '1',
 'morph': 'N-NDF/N-VDF/N-ADF',
 'number': 'dual',
 'pos': 'noun',
 'raw_bc': '*di/a',
 'raw_uc': 'Δία',
 's

In [20]:
pp.pprint(morphkit.analyse_word_with_morpheus("puella",api_endpoint,language='latin',debug=True))

[analyse_word_with_morpheus] Calling function get_word_blocks(word_beta='puella',api_endpoint='10.0.1.156:1315',language='latin',debug=True,timeout=None,retry_attempts=None,retry_delay=None)
[get_word_blocks] Sending GET request: http://10.0.1.156:1315/latin/puella?opts=d?opts=n
[get_word_blocks] Received status code: 200
[get_word_blocks] Response time: 0.097s
[get_word_blocks] Request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
[get_word_blocks] Response headers: {'Content-Type': 'text/html;charset=utf-8', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Length': '252'}
[get_word_blocks] Response snippet (max 100 bytes):

:raw puella

:workw puella_
:lem puella
:prvb 				
:aug1 				
:stem puell	 fem			a_ae
:suff 				
:e...
[split_into_raw_blocks] function called
Received 2 raw blocks
[analyse_word_with_morpheus] Calling fu

In [21]:
morphkit.analyse_word_with_morpheus("kai\\",api_endpoint,debug=False)

{'raw_bc': 'kai\\',
 'raw_uc': 'καὶ',
 'blocks': 1,
 'analyses': [{'raw_bc': 'kai\\',
   'raw_uc': 'καὶ',
   'workw_bc': 'kai/',
   'workw_uc': 'καί',
   'lem_full_bc': 'kai/',
   'lem_full_uc': 'καί',
   'lem_base_bc': 'kai/',
   'lem_base_uc': 'καί',
   'stem_bc': 'kai/',
   'stem_uc': 'καί',
   'stem_flags': ['indeclform'],
   'end_flags': ['indeclform'],
   'end_codes': ['conj'],
   'pos': 'conjunction',
   'morph': 'CONJ'}]}

# 6 - Test timeout to non-existing API endpoint <a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

In [22]:
raw_text=morphkit.get_word_blocks(bc_word,"10.0.1.150:1315",debug=True)

[get_word_blocks] Sending GET request: http://10.0.1.150:1315/greek/%2Adi%2Fa?opts=d?opts=n
[get_word_blocks] Timeout; retry 1/3
[get_word_blocks] Timeout; retry 2/3
[get_word_blocks] Timeout; retry 3/3


MorpheusTimeoutError: Request timed out after 30 seconds (attempts: 4).

In [ ]:
raw_text=morphkit.get_word_blocks(bc_word,"10.0.1.150:1315",debug=False)

MorpheusTimeoutError: Request timed out after 30 seconds (attempts: 4).

# Test optional paramater

In earlier runs responce times are reported like:
<pre>
[get_word_blocks] Response time: 0.037s
</pre>
To force timeout scenairo, we will set the timeout to be significantly lower.

In [ ]:
bc_word=beta_code.greek_to_beta_code(u'Δία')
api_endpoint="10.0.1.156:1315" # IP/port
raw_text=morphkit.get_word_blocks(bc_word,api_endpoint,timeout=0.005,debug=True)

[get_word_blocks] Sending GET request: http://10.0.1.156:1315/greek/%2Adi%2Fa?opts=d?opts=n
[get_word_blocks] Timeout; retry 1/3
[get_word_blocks] Received status code: 200
[get_word_blocks] Response time: 0.042s
[get_word_blocks] Request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
[get_word_blocks] Response headers: {'Content-Type': 'text/html;charset=utf-8', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Length': '1759'}
[get_word_blocks] Response snippet (max 100 bytes):

:raw *di/a

:workw *di/a_
:lem *di/h
:prvb 				
:aug1 				
:stem *di	 fem sg			h_hs
:suff 				
:end...


In [ ]:
raw_text=morphkit.get_word_blocks(bc_word,api_endpoint,timeout=0.0001,debug=True)

[get_word_blocks] Sending GET request: http://10.0.1.156:1315/greek/%2Adi%2Fa?opts=d?opts=n
[get_word_blocks] Timeout; retry 1/3
[get_word_blocks] Timeout; retry 2/3
[get_word_blocks] Timeout; retry 3/3


MorpheusTimeoutError: Request timed out after 0.0001 seconds (attempts: 4).

# 5 - Notebook version<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>January 14, 2026</td>
    </tr>
  </table>
</div>